In [1]:
from github import Github

url = "HikaGenji/hl-dl"
g = Github()  # No authentication needed for public repos, but you can add a token for higher rate limits
repo = g.get_repo(url)
contents = repo.get_contents("data")
files = [content_file.path for content_file in contents]

In [4]:
import duckdb

parquet_url = f"https://github.com/{url}/raw/main/{files[0]}"

# Example query: select columns and filter rows
result = duckdb.query(f"""
    SELECT *
    FROM '{parquet_url}'
    LIMIT 100
""").to_df()

print(result.head())


                    timestamp                                  ethAddress  \
0  2025-06-07T12:36:00.763501  0x162cc7c861ebd0c06b3d72319201150482518185   
1  2025-06-07T12:36:00.763543  0x7b7f72a28fe109fa703eeed7984f2a8a68fedee2   
2  2025-06-07T12:36:00.763563  0xff4cd3826ecee12acd4329aada4a2d3419fc463c   
3  2025-06-07T12:36:00.763582  0xecb63caa47c7c4e77f60f1ce858cf28dc2b82b00   
4  2025-06-07T12:36:00.763594  0x5078c2fbea2b2ad61bc840bc023e35fce56bedb6   

   accountValue           displayName  prize        day_pnl       day_roi  \
0  2.393132e+07                   ABC      0  377343.541175  1.336961e-02   
1  4.290938e+06  beatmaker1@proton.me      0  214309.023703  5.740059e-02   
2  2.564262e+07                  None      0  594199.911583  2.873628e-02   
3  1.062553e+08                  None      0 -780001.125411 -1.364246e-02   
4  7.109793e+00    JamesWynn $moonpig      0       0.410996  5.814000e-07   

     day_volume      week_pnl  week_roi   week_volume     month_pnl  \
0  